In [3]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import norm


In [4]:
df = pd.read_excel('spx_quotedata20220308_all.xlsx')
pd.set_option('display.max_columns', 25)
df["Expiration Date"] = pd.to_datetime(df["Expiration Date"])
df


,Expiration Date,Calls,Last Sale,Net,Bid,Ask,Volume,IV,Delta,Gamma,Open Interest,Strike,Puts,Last Sale.1,Net.1,Bid.1,Ask.1,Volume.1,IV.1,Delta.1,Gamma.1,Open Interest.1
0,2022-03-09,SPXW220309C03000000,0.00,0.00,1151.7,1168.9,0,4.3394,1.0000,0.0000,0,3000,SPXW220309P03000000,0.05,0.0000,0.0,0.05,6,1.7782,-0.0001,0.0000,2172
1,2022-03-09,SPXW220309C03200000,0.00,0.00,951.7,969.1,0,3.6439,0.9999,0.0000,0,3200,SPXW220309P03200000,0.05,-0.0250,0.0,0.05,4,1.4385,-0.0001,0.0000,6683
2,2022-03-09,SPXW220309C03300000,0.00,0.00,851.2,869.3,0,3.3092,0.9999,0.0000,0,3300,SPXW220309P03300000,0.05,-0.0250,0.0,0.05,2182,1.2743,-0.0002,0.0000,6884
3,2022-03-09,SPXW220309C03400000,0.00,0.00,751.7,769.1,0,2.9763,0.9998,0.0000,0,3400,SPXW220309P03400000,0.05,-0.0750,0.0,0.05,360,1.1260,-0.0002,0.0000,13154
4,2022-03-09,SPXW220309C03500000,735.06,0.00,651.2,669.4,0,2.6471,0.9998,0.0000,10,3500,SPXW220309P03500000,0.05,-0.1250,0.0,0.05,1264,0.9697,-0.0003,0.0000,10544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5886,2026-12-18,SPX261218C07200000,115.30,0.00,34.3,114.3,0,0.1763,0.1306,0.0001,12,7200,SPX261218P07200000,2623.60,-59.1000,2602.4,2794.40,80,0.1427,-0.8879,0.0001,80
5887,2026-12-18,SPX261218C08000000,51.57,0.00,0.0,300.0,0,0.1791,0.0851,0.0001,572,8000,SPX261218P08000000,3334.08,-46.3199,3299.5,3491.50,3,0.0010,-0.9333,0.0001,4
5888,2026-12-18,SPX261218C08600000,33.22,0.00,0.0,300.0,0,0.1822,0.0637,0.0001,1,8600,SPX261218P08600000,3862.26,-52.7400,3832.4,4024.40,5,0.0010,-0.9548,0.0001,5
5889,2026-12-18,SPX261218C09000000,35.66,0.00,0.0,300.0,0,0.1844,0.0529,0.0001,235,9000,SPX261218P09000000,4236.09,-38.1102,4190.1,4382.10,2,0.0010,-0.9656,0.0001,2


In [5]:
# Call price as middle of bid and ask
df["Mid"] = (df["Bid"] + df["Ask"]) / 2

# Put price
df["Mid.1"] = (df["Ask.1"] + df["Bid.1"])/2

S0 = 4170.7002

# List of unique maturities
maturities = df["Expiration Date"].unique()

# Define model function as call put indentity
def model(x,*args):
    S0,df_mat = args
    call_put_identity = df_mat["Mid"] - df_mat["Mid.1"] - x[0] * S0 + x[1] * df_mat["Strike"]
    return call_put_identity.values

# Define squared deviation function
def squared_deviation(x,*args):
    call_put_indentities = model(x,*args)
    return np.sum((call_put_indentities- np.mean(call_put_indentities))**2)

B0 = [0]
D0 = [0]
for maturity in maturities:
    #Ensure that we have positive zero coupon bond prices and dividend discount factors
    df_mat = df[df["Expiration Date"] == maturity]
    cons = [
        # Ensure that we have positive zero coupon bond prices and dividend discount factors
        {'type': 'ineq', 'fun': lambda x:  x[0] },
        {'type': 'ineq', 'fun': lambda x:  x[1] },

        # Add constraint to ensure that the sum of the call put identities is close to 0, necessary for the optimization to converge
        {'type': 'eq', 'fun': lambda x:  np.sum(model(x,S0,df_mat)) }
    ]   

    
    # Minimize squared deviation
    m = minimize(squared_deviation,[D0[-1],B0[-1]],args=(S0,df_mat),constraints=cons,method='SLSQP')

    D0.append(m.x[0])
    B0.append(m.x[1])

# Remove initial values
B0 = B0[1:]
D0 = D0[1:]
print("B0: ",B0)
print("D0: ",D0)


B0:  [0.9999651962556683, 1.000078698449191, 1.0001504905726888, 0.9998474781854488, 0.9999272888853734, 1.0002223863279256, 1.0003459326381765, 0.9998666573263187, 0.9999690925031743, 1.0000626251443314, 1.000009874286806, 1.000075530486597, 0.9999529754037009, 1.0004910938255864, 1.0000660377217732, 1.0000169010742936, 0.9998446821735724, 0.9995781225890411, 0.9991721287656612, 0.9990067400893321, 0.9990067400893321, 0.9981014227844333, 0.9978651722676555, 0.9966823239674072, 0.996276560674498, 0.9964754631128985, 0.9955870176443226, 0.9943146105728118, 0.9940273304823528, 0.9927207481607546, 0.9899418349368542, 0.9900441998397024, 0.988629456730605, 0.9832642588221041, 0.9835992933342884, 0.9835992933342884, 0.9701744619964902, 0.9701744619964902, 0.9701744619964902, 0.9701744619964902]
D0:  [0.9974955036150942, 0.9974617628107693, 0.9972630879390384, 0.9970302487454683, 0.9969447144056025, 0.9972515140948462, 0.9973268581260049, 0.9968310945910458, 0.9969125530699493, 0.99695186645

In [6]:
forward_prices = [S0 / B0 * D0 for B0,D0 in zip(B0,D0)]
print("Forward prices: ",forward_prices)

Forward prices:  [4160.399493906878, 4159.786604892458, 4158.659521266993, 4158.948588234078, 4158.259871470552, 4158.31233747458, 4158.112899686276, 4158.038089493216, 4157.951896356386, 4157.72697055771, 4157.688069497232, 4157.497154119908, 4157.645039780674, 4156.81871538892, 4156.2867029284525, 4156.311290149056, 4156.17425085928, 4156.171951059859, 4152.961705838964, 4151.991942426174, 4151.991942426174, 4150.193499016367, 4150.904956288726, 4151.5641336445315, 4151.002057527836, 4150.568813726582, 4150.125137425059, 4154.190508438367, 4153.151340563798, 4153.6562578809035, 4154.341919340757, 4159.146433592798, 4162.247310938586, 4165.342045685644, 4164.587353845361, 4164.587353845361, 4200.336801517922, 4200.336801517922, 4200.336801517922, 4200.336801517922]


$$
S(T) = S_0 \exp( \int_0^T ( r(t) -q(t) - \frac{1}{2} \sigma(t)^2 ) dt + \int_0^T \sigma(t) dW(t) )
\\
But
\\
\sigma(t)\:is\:a\:piecewise\:function\:so
\\
\int_0^T \sigma(t) dt = \sum_{i=1}^{n} \int_{T_{i-1}}^{T_i} \sigma_i(t)
\\
Moreover\:\sigma_i(t)\:is\:constant\:on\:the\:intervals\:[T_{i-1},T_{i}]
\\
Then
\\
\sum_{i=1}^{n} \int_{T_{i-1}}^{T_i} \sigma_i(t) = \sum_{i=1}^{n} \sigma_i (T_{i}-T_{i-1})
\\
And
\\
\exp(-\int_0^T r(t) dt) = B(0,T)
\\
Then
\\
S(T) = S(0) \frac{D(0,T)}{B(0,T)} \exp(-\frac{1}{2} \sum_{i=1}^{n} \sigma_i^2 (T_{i}-T_{i-1}) + \sum_{i=1}^{n} \sigma_i \int_{T_{i-1}}^{T_i} dW(t) )
\\
However\:the\:payoff\:for\:an\:European\:call\:option\:is
\\
h_1, h_2 = \frac{\ln\left(\frac{S(0)}{K}\right) + \ln(\frac{D(0,T)}{B(0,T)}) \pm \frac{1}{2} \sum_{i=1}^{n} \sigma_i^2 (T_{i}-T_{i-1})}{\sqrt{ \sum_{i=1}^{n} \sigma_i^2  (T_{i}-T_{i-1})T}}
\\
Finally
\\
C = S(0)D(0,T)N(h_1)-KB(0,T)N(h_2)


$$


In [9]:
closest_strike = []

for forward_price in forward_prices:
    closest_strike.append(df.iloc[(df["Strike"]-forward_price).abs().argsort()[:1]]["Strike"].values[0])
print("forward prices: ",forward_prices)
print("Closest strike: ",closest_strike)

forward prices:  [4160.399493906878, 4159.786604892458, 4158.659521266993, 4158.948588234078, 4158.259871470552, 4158.31233747458, 4158.112899686276, 4158.038089493216, 4157.951896356386, 4157.72697055771, 4157.688069497232, 4157.497154119908, 4157.645039780674, 4156.81871538892, 4156.2867029284525, 4156.311290149056, 4156.17425085928, 4156.171951059859, 4152.961705838964, 4151.991942426174, 4151.991942426174, 4150.193499016367, 4150.904956288726, 4151.5641336445315, 4151.002057527836, 4150.568813726582, 4150.125137425059, 4154.190508438367, 4153.151340563798, 4153.6562578809035, 4154.341919340757, 4159.146433592798, 4162.247310938586, 4165.342045685644, 4164.587353845361, 4164.587353845361, 4200.336801517922, 4200.336801517922, 4200.336801517922, 4200.336801517922]
Closest strike:  [4160, 4160, 4160, 4160, 4160, 4160, 4160, 4160, 4160, 4160, 4160, 4155, 4160, 4155, 4155, 4155, 4155, 4155, 4155, 4150, 4150, 4150, 4150, 4150, 4150, 4150, 4150, 4155, 4155, 4155, 4155, 4160, 4160, 4165, 4